In [1]:
import sys
sys.path.append("../")
import numpy as np 
from utils import *
from lattice import *
import pandas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensornetwork as tn

In [153]:
sps = 3
bd = 2
seed = 36
A = FF.block1D(bd, sps, bd, seed=seed, canonical=True).real
tmp = np.zeros((2,2))
for i in range(2):
    tmp += A[:,i,:] @ A[:,i,:].T
print(tmp)

[[ 0.93465598 -0.22428086]
 [-0.22428086  0.12754249]]


In [154]:
A2 = np.einsum("ijk,klm->jlim", A, A).reshape(sps**2, bd**2)
U, s , V = np.linalg.svd(A2)
Up = U[:, len(s):]
h = Up @ Up.T

In [155]:
mps_nodes, mps_edges = FF.create_MPS(6, A)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y1 = C.tensor.reshape(-1)
y1 /= np.linalg.norm(y1)

In [156]:
y_ = y1.reshape((sps**2, -1))
rho = y_ @ y_.T 
print("y is eigen state for h? : ",np.trace(rho @ h))

y is eigen state for h? :  9.312774901445156e-18


### Check canonical give same state

In [157]:
B = FF.block1D(bd, sps, bd, seed=seed, canonical=False).real
B2 = np.einsum("ijk,klm->jlim", B, B).reshape(sps**2, bd**2)
U, s , V = np.linalg.svd(B2)
Up = U[:, len(s):]
hb = Up @ Up.T
mps_nodes, mps_edges = FF.create_MPS(6, B)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y2 = C.tensor.reshape(-1)
y2 /= np.linalg.norm(y2)
np.linalg.norm(y1 - y2), np.linalg.norm(h - hb)

(5.114424018633868e-16, 2.3865583919308875e-15)

### Check energy gap

In [158]:
TA = np.einsum("jik,lim->jlkm", A, A).reshape(bd**2, bd**2)
e_ = np.sort(np.linalg.eigvals(TA))
gap_transfer = e_[-1] - e_[-2]
gaps = []
for L in range(2, 8):
    H = sum_ham(h, [[i, (i + 1) % L] for i in range(L)], L, sps)
    E = np.linalg.eigvalsh(H)
    gap = E[1] - E[0]
    gaps.append(gap)

In [159]:
gaps

[3.082995880203112e-16,
 0.0043482878883649555,
 0.004688357437257469,
 0.0049369975793628115,
 0.005064493273186074,
 0.0051235460386071805]

In [86]:
gap_transfer

0.5362674250844774

In [75]:
np.linalg.eigvalsh(h)

array([-1.59633129e-17,  8.29053885e-17,  2.01326062e-16,  2.51122624e-16,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00])